# Creating a Master Dataset
So now we can merge them all into several large data files - separated by year.

In [86]:
# Import the relevant packages
import pandas as pd
import datetime as dt
import glob
import numpy as np

In [87]:
# Start by merging the shark detection data with the specifics of the shark tagging data
shark_detect = pd.read_csv('D:/Documents/SpringBoard/capstone-1/datasets/edited_files/shark_detection_data2.csv')
    # read in this dataset
shark_detect = shark_detect[['Transmitter', 'Zone', 'Date', 'Lat', 'Lng']] 
    # only keep these columns
shark_detect = shark_detect.astype({'Transmitter':'category', 'Zone':'int64', 
                                    'Date':'category', 'Lat':'float64', 'Lng':'float64'})
    # And make sure the types are appropraite

In [88]:
shark_meta = pd.read_csv('D:/Documents/SpringBoard/capstone-1/datasets/tagged_sharks.csv')
    # read in the shark tagging deployment datasheet
shark_meta = shark_meta[['Transmitter', 'animal_weight', 'animal_length_total', 'gender', 'location', 'year']]
    # keep these values
shark_meta = shark_meta.astype({'Transmitter':'category', 'animal_weight': 'float64', 
                                'animal_length_total':'float64', 'gender':'category', 
                                'location':'category', 'year':'category'})
    # And make sure the types are appropraite

In [89]:
# Merge the shark detection data with the transmitter deployment data and reset the index
shark_all = shark_detect.merge(shark_meta, how='inner').reset_index(drop=True)
    # we do an inner join because we only want to keep data that are present in both datasets

In [90]:
# Now let's merge the number of receivers per day
rec_day = pd.read_csv('D:/Documents/SpringBoard/capstone-1/datasets/final_files/receiver_density_2.csv')
    # read in the receiver density files
rec_day = rec_day[['Date', 'Receiver_D', 'Zone']] # only keep the relevant columns
rec_day['Date'] = rec_day['Date'].str.split(' ', expand=True, n=1)[0] # make sure the date is in the proper format
#rec_day['Zone'] = rec_day['Zone'].astype('str') # make sure the zone is a string
#rec_day['Zone'] = rec_day['Zone'].str.split('.', expand=True, n=1)[0] # and get rid of the silly .0
# rec_day = rec_day.astype('category') # all values can be considered categories at this point

In [91]:
# And merge all the shark detection data with the receiver data and reset the index
shark_all = shark_all.merge(rec_day, how='outer').reset_index(drop=True)
    # This time we do an outer merge because we want to keep all data, 
    # even if no sharks were detected

In [71]:
shark_all.head()

,Transmitter,Zone,Date,Lat,Lng,animal_weight,animal_length_total,gender,location,year,Receiver_D
268175,NaN,1291014,2019-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
268176,NaN,1291015,2019-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
268177,NaN,1296043,2019-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
268178,NaN,1298528,2019-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
268179,NaN,1198628,2019-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [92]:
# Ok so now we can merge other data... lets do the environmental data by year... 
dep = pd.read_csv('D:/Documents/SpringBoard/capstone-1/datasets/final_files/seafloor_depth_gradient.csv')
    # the depth gradient data will not change so we will read it in outside of the loop
dep = dep[['Zone', 'm']] # we only really need the zone and the depth value
dep.columns = ['Zone', 'DepthGradient'] # and we can rename the columns
dep = dep.astype({'Zone':'int64', 'DepthGradient':'float64'})

In [81]:
# Ok so now we can merge other data... lets do the environmental data by year... 
#dep = pd.read_csv('D:/Documents/SpringBoard/capstone-1/datasets/final_files/seafloor_depth_gradient.csv')
    # the depth gradient data will not change so we will read it in outside of the loop
#dep = dep[['Zone', 'm']] # we only really need the zone and the depth value
#dep.columns = ['Zone', 'DepthGradient'] # and we can rename the columns
#dep = dep.astype({'Zone':'int64', 'DepthGradient':'float64'})

years = range(2012, 2020) # this is the range of years that we have, 2012 to 2019
#years = range(2012,2013)

shark_all['DetYear'] = pd.to_datetime(shark_all.Date).dt.year
    # let's change the date value to detection year, so that we can parse these data into years and
    # hopefully speed up our data processing.

for year in years: # for each year
    shark_temp = shark_all[shark_all.DetYear == year] # subset the detection data
    env_files = glob.glob('D:/Documents/SpringBoard/capstone-1/datasets/final_files/*'+str(year)+'*')
        # and get a list of environmental files that are relevant to that year
    for file in env_files: # for each environmental file
        temp_env = pd.read_csv(file) # read in the file
        temp_env['deg_N_'+str(temp_env.columns[-2])] = temp_env['degrees_north']
            # make a latitude column that is unique to that environmental dataset
        temp_env['deg_E_'+str(temp_env.columns[-3])] = temp_env['degrees_east']
            # make a longitude column that is unique to that environmental dataset
        these_cols = temp_env.columns[[0,-1,-2,-3,-4]] # now, we only want the date column, 
            # the zone information (-1), the environmental information (-2), and the unique
            # latitude (-3), and longitude (-4) columns
        temp_env = temp_env[these_cols] # let's just keep those columns in the dataset
        temp_env['Date'] = temp_env['UTC'].str.split('T', expand=True, n=1)[0] # and we want
            # to split the UTC values into just days (they have a weird T present)
        temp_env = temp_env.drop(labels='UTC', axis=1) # we want to drop the UTC column and
            # just keep the days
        temp_env = temp_env.astype('category') # all values can be considered categories
        temp_env = temp_env.astype({'Zone': 'int64'})
        if file == env_files[0]: # if this is the first file in the list
            comb_dat = temp_env.merge(shark_temp, how='outer') # merge it with the shark data itself
        else: # but if it's any subsequent file
            comb_dat = temp_env.merge(comb_dat, how='outer') # merge it with the already merged list
                # these merges will go by zone and date only bc those are the two columns that should
                # be the same across each dataframe
    comb_dat = dep.merge(comb_dat, how='right').reset_index(drop=True) # merge again with the depth
        # data and make sure to reset the index
    comb_dat.to_csv('D:/Documents/SpringBoard/capstone-1/datasets/final_files/combined/'
                    +str(year)+'-without_moon2.csv', index=False)
        # and save the data each year
        # moon data will come later using pylunar.

In [93]:
# So now, we want to fill in the data gaps where the environmental data were NAs
files_fill = glob.glob('D:/Documents/SpringBoard/capstone-1/datasets/final_files/combined/*2.csv')
    # first lets get a list of the fiels that we want to iterate through

In [95]:
for file in files_fill: # for each of these files
    comb_dat = pd.read_csv(file) # read in the file
    
    # Set up
    N_cols = ['deg_N_degree_C', 'deg_N_PSU', 'deg_N_mg m^-3', 'Lat'] 
        # save the column names for latitude
    E_cols = ['deg_E_degree_C', 'deg_E_PSU', 'deg_E_mg m^-3', 'Lng']
        # save the column names for longitude
        
    # Calculate
    lats = comb_dat.loc[:, N_cols] # grab the latitudes of all environmental datasets
    lats_mean = lats.apply(np.mean, axis=1) # and get the mean of all latitudes for each row
    lngs = comb_dat.loc[:, E_cols] # grab all the longitudes of all environmental datasets
    lngs_mean = lngs.apply(np.mean, axis=1) # and get the mean of all longitudes for each row
    
    # Clean up
    comb_dat = comb_dat.drop(columns=N_cols) # remove the excess longitude columns
    comb_dat = comb_dat.drop(columns=E_cols) # remove the excess latitude columns
    
    # Update
    comb_dat['Lat'] = lats_mean # make the latitude value the mean latitude for each row
    comb_dat['Lng'] = lngs_mean # make the longitude value the mean longitude for each row
    
    # Sort
    comb_dat = comb_dat.sort_values(by=['Date','Lat', 'Lng']) # order the data by date, lat, then lon
    comb_dat = comb_dat.dropna(subset=['Lat', 'Lng']) # and remove any lat/lngs that are NA's
        # it means there's literally no geospatial data associated with these zones, so they're irrelevant
    
    # Fill NAs
    nans = comb_dat[['DepthGradient', 'degree_C', 'PSU', 'mg m^-3']].fillna(method='bfill')
        # So we have to do a double fill here, because for most of the data the backfill method will work
        # so we do that first for all the environmental data values that are na... 
        # we save it to a new dataframe
    nans = nans[['DepthGradient', 'degree_C', 'PSU', 'mg m^-3']].fillna(method='ffill')
        # But the very end of the dataset cannot be backfilled (there's nothing in front of it to fill)
        # so we'll do a front fill for the very end
        # Now there should be no NA values
    comb_dat['Receiver_D'] = comb_dat['Receiver_D'].fillna(0) # we also need to remember to fill 
        # NAs where there are no acoustic receivers present; there are no data because there are 0
        # receivers in the water, so we can just fill NAs with 0
    
    # Clean up
    comb_dat = comb_dat.drop(columns = ['DepthGradient', 'degree_C', 'PSU', 'mg m^-3'])
        # let's drop these messy NA filled columns... 
    comb_dat['DepthGradient'] = nans['DepthGradient'] # and replace them with the non-NA datasets
    comb_dat['TempC'] = nans['degree_C'] # We will replace them with names that are more intuitive
    comb_dat['Sal'] = nans['PSU'] # salinity
    comb_dat['ChlA'] = nans['mg m^-3'] # chlorophyll a
    comb_dat = comb_dat.drop(columns=['DetYear']) # we don't need detection year anymore
    comb_dat = comb_dat.reset_index(drop=True) # and we want to reset the index
    
    # Save it
    filename = file.split('\\')[1] # grab the file name from the list of files
    comb_dat.to_csv('D:/Documents/SpringBoard/capstone-1/datasets/final_files/combined/filled/'+filename, index=False)
        # and save it.

And now, all that's left is to add the moon phase data... phew.
Go to the Adding Moon Phases notebook next. 